# 1. Importing Libraries
---

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ML Libraries
from sklearn.metrics import classification_report, f1_score, confusion_matrix
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
import optuna
from optuna.pruners import HyperbandPruner
from sklearn.model_selection import cross_val_score,train_test_split, StratifiedKFold
from scipy.stats import randint, uniform, loguniform

# Avoid warnings to show-up
import warnings
warnings.filterwarnings('ignore')

# 2. Data Investigation
---

In [ ]:
raw = pd.read_csv('covtype.csv')
raw

In [ ]:
# There are 581012 rows and 55 columns in the dataset
# All columns are numerical
# It seems that there are no missing values in the dataset
# It is a big dataset with many features

raw.info()

In [ ]:
# There are too many classes that are One Hot Encoded (OHE), I wonder if there are overlapping entries in these columns
# Let's separate OHE columns and the rest of the columns
soil_cols = [col for col in raw.columns if 'Soil_Type' in col]
wilderness_cols = [col for col in raw.columns if 'Wilderness_Area' in col]

ohe_cols = soil_cols + wilderness_cols

non_ohe_cols = [col for col in raw.columns if col not in ohe_cols]

# 3. Exploratory Data Analysis
---

In [ ]:
# Let's see if there are overlapping entries in Soil_Type columns
# This is to check if any row has more than one Soil_Type marked as 1

for i, col in  enumerate(soil_cols):
    if raw[raw[col] == 1][soil_cols].sum().sum() > raw[raw[col] == 1][soil_cols].sum().iloc[i]:
        print(f'Warning: {col} has overlapping entries with other Soil_Type columns.')
    else:
        pass

# It seems that there are no overlapping entries in Soil_Type columns
# Hence, each row has only one Soil_Type marked as 1

In [ ]:
# Let's see if there are overlapping entries in Wilderness_Area columns
# This is to check if any row has more than one Wilderness_Area marked as 1

for i, col in  enumerate(wilderness_cols):
    if raw[raw[col] == 1][wilderness_cols].sum().sum() > raw[raw[col] == 1][wilderness_cols].sum().iloc[i]:
        print(f'Warning: {col} has overlapping entries with other Wilderness_Area columns.')
    else:
        pass

# It seems that there are no overlapping entries in Wilderness_Area columns
# Hence, each row has only one Wilderness_Area marked as 1

In [ ]:
# Let's check the distribution of the target variable 'Cover_Type'
# It is a multi-class classification problem with 7 classes
# The classes are imbalanced, therefore we need to use stratified sampling

sns.barplot(x=raw['Cover_Type'].value_counts().index, y=raw['Cover_Type'].value_counts().values)

In [ ]:
# Let's visualize the distribution of Elevation for each Cover_Type using boxenplot
plt.figure(figsize=(15,6))
sns.boxenplot(data=raw,y='Elevation',x='Cover_Type')

# It seems that Elevation plays a role in determining some of Cover_Types
# For example, Cover_Type 1 and 2 are generally found at higher elevations compared to Cover_Type 5 and 6

In [ ]:
# We saw that Cover_Type has some relation with Elevation
# Now, let's check the relationship between Soil_Type and Elevation 
# I will also include Cover_Type in the visualization to see if there are any patterns

# Since the dataset is already One-Hot Encoded, we need to convert the one-hot encoded columns back to a single categorical column 'Soil_Type_Cat'
# idxmax will return the column name with the maximum value (which is 1 in this case) for each row (since there are no overlapping entries)
# I will then extract only the number from the column name to create a new column 'Soil_Type_Cat' (to make it easier to visualize)
raw['Soil_Type_Cat'] = raw[soil_cols].idxmax(axis=1).str.replace('Soil_Type', '').astype(int)

# Now, let's create the stripplot of Elevation for each Soil Type with Cover_Type as hue
plt.figure(figsize=(20, 10))
sns.stripplot(data=raw, x='Soil_Type_Cat', y='Elevation', hue='Cover_Type', alpha=0.6, size=3, jitter=0.3, palette='Dark2')
plt.title('Elevation Distribution by Soil Type', fontsize=16)
plt.xlabel('Soil Type')
plt.ylabel('Elevation')
plt.xticks(rotation=90)
plt.legend(title='Cover_Type', ncol=7, markerscale=5, fontsize=10, title_fontsize=10, )
plt.tight_layout()


# Soil Type is indeed related to Elevation and Cover Type
# Some soil types are exclusive to certain cover types, indicating a strong relationship between soil composition and vegetation type
# As we have already seen in the boxenplot, Cover_Type 1, 2, and 7 are generally found at higher elevations share common Soil_Types (especially Soil_Type >20)
# Cover_Type 3 and 4 are found at lower eleveations and also share common Soil_Types (especially Soil_Type <17)
# Ordinal Encoding might be really good for Soil_Type instead of One-Hot Encoding to reduce dimensionality and capture the inherent order in soil types based on elevation and cover type relationships

In [ ]:
raw['Wilderness_Cat'] = raw[wilderness_cols].idxmax(axis=1).str.replace('Wilderness_Area', '').astype(int)

# Now, let's create the stripplot of Elevation for each Wilderness Area with Cover_Type as hue
plt.figure(figsize=(16, 10))
sns.stripplot(data=raw, x='Wilderness_Cat', y='Elevation', hue='Cover_Type', alpha=0.6, size=3, jitter=0.1, palette='Dark2')
plt.title('Elevation Distribution by Wilderness Area', fontsize=16)
plt.xlabel('Wilderness Area')
plt.ylabel('Elevation')
plt.xticks(rotation=90)
plt.legend(title='Cover_Type', ncol=7, markerscale=5, fontsize=10, title_fontsize=10, )
plt.tight_layout()

# Wilderness Area does not seem to have a strong relationship with Elevation
# However, some Wilderness Areas do show a tendency to be associated with certain Cover Types
# For example, Wilderness Area 1 seems to have a higher concentration of Cover_Type 1, Cover_Type 2, and Cover_Type 7 (very low or none of Cover_Type 3, 4, 5, 6)
# Wilderness Area 3 seems to have a broader distribution of Cover_Types, there is even a clear separation of some parts of Cover_Type 7 as the elevation increases, 
#   this area has the broadest cover of elevations, hence provides rich biodiversity unlike Wilderness Area 1
# Wilderness Area 4 seems to be associated with lower elevations and has a higher concentration of Cover_Type 1 and Cover_Type 7
# Since there is no clear relationship in between Wilderness Area and Elevation, One-Hot Encoding seems to be a good choice unlike Soil_Type columns

# Hence, I will drop the Wilderness_Cat column
raw.drop(columns='Wilderness_Cat',axis=1,inplace=True)

In [ ]:
# Let's see if there are any correlations between the features

non_ohe_cols = [col for col in raw.columns if col not in ohe_cols] # Update non_ohe_cols to include the new features 

plt.figure(figsize=(12,12))
sns.heatmap(raw[non_ohe_cols].corr(), annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.8, annot_kws={"size": 10})
plt.title('Non-OHE Correlation Matrix', fontsize=16)
plt.tight_layout()

# There are some correlations between the non-OHE features, however, although it is notable that there is a mildly strong correlation between Hillshade_9am and Hillshade_Noon (0.78), it is better to keep them both for logistic reasons
# Similarly, the correlation between Aspect and Hillshade features are not weak, but it is logical to keep them both as they represent different concepts for the same entity of Cover_Type
# There is no strong correlation between the non-OHE features and the target variable 'Cover_Type'
# As we have suspected there is a positive correlation between Elevation and Soil_Type class numbering

In [ ]:
# Let's drop the One-hot encoded Soil_Types)

print(f'Shape before dropping Soil_Type columns: {raw.shape[1]}')
raw.drop(columns=soil_cols, axis=1, inplace=True)
print(f'Shape after dropping Soil_Type columns: {raw.shape[1]}')

In [ ]:
# Let's visualize the distribution of the non-OHE features
# There are both right and left skewed features in the dataset
# These features might need transformation before feeding them into the model for non-Tree based algorithms to both ease the training and to satisfy the assumptions of the algorithms
# However, I will be using tree-based algorithms, so I can skip the transformations for now as they are not sensitive to feature distributions

fig, axes = plt.subplots(4, 3, figsize=(16,16))
axes = axes.flatten()

for i, feature in enumerate(non_ohe_cols):
    sns.histplot(
        data=raw,
        x=feature,
        multiple="dodge",
        shrink=0.9,
        ax=axes[i],
        bins=50
    )
    axes[i].set_title(feature)

plt.tight_layout()
plt.show()

# 4. Data Preprocessing
---


* Since I will use tree based models for such a large dataset, there is no restriction to use Gaussian-like distribution of continuous features
* Categorical variables are either one-hot encoded, or label encoded
* Therefore, there will be no extra data preprocessing required for this dataset.

Classfication Algorithms to be used:

- Random Forest
- XGBoost
- LightGBM

# 5. Modelling
---

In [ ]:
# Let's separate features and target variable with stratified sampling

X = raw.drop('Cover_Type', axis=1)
y = raw['Cover_Type'] - 1 # Making classes start from 0

# Let's split the data into training and testing sets with stratification as the target variable is imbalanced
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# I will use Optuna to perform hyperparameter optimization for three different models: RandomForest, XGBoost, and LightGBM
# The objective function required by Optuna will be defined to handle the hyperparameter tuning for each model
# Hyperparameter ranges are defined based on first common practices and then are fine tuned based on quick validation results of the ranges for specific characteristics of each model (with Optuna visualization tools)


def objective(trial, model_name, X, y):
    if model_name == 'RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 100, 1000)
        max_depth = trial.suggest_categorical('max_depth', [None, 10, 30, 50, 60])
        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=42)

    elif model_name == 'XGBoost':
        n_estimators = trial.suggest_int('n_estimators', 400, 600)
        max_depth = trial.suggest_int('max_depth', 6, 35)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.1, log=True)
        subsample = trial.suggest_categorical('subsample', [0.6, 0.7, 0.8, 0.9, 1.0])
        colsample_bytree = trial.suggest_categorical('colsample_bytree', [0.6, 0.7, 0.8, 0.9, 1.0])
        reg_alpha = trial.suggest_float('reg_alpha', 1e-2, 1e0, log=True)
        reg_lambda = trial.suggest_float('reg_lambda', 1e-2, 1e0, log=True)
        gamma = trial.suggest_float('gamma', 1e-3, 0.5, log=True)
        objective = 'multi:softmax'
        booster = trial.suggest_categorical('booster', ['gbtree'])
        tree_method = trial.suggest_categorical('tree_method', ['gpu_hist'])
        model = XGBClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            subsample=subsample,
            colsample_bytree=colsample_bytree,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            gamma=gamma,
            objective=objective,
            tree_method=tree_method,
            booster=booster,
            random_state=42,
            n_jobs=-1,
            verbosity=0)

    elif model_name == 'LightGBM':
        n_estimators = trial.suggest_int('n_estimators', 500, 1000)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.1, log=True)
        num_leaves = trial.suggest_int('num_leaves', 20, 75)
        bagging_fraction = trial.suggest_categorical('bagging_fraction', [0.6, 0.7, 0.8, 0.9])
        feature_fraction = trial.suggest_categorical('feature_fraction', [0.6, 0.7, 0.8, 0.9])
        reg_alpha = trial.suggest_float('reg_alpha', 1e-2, 1e0, log=True)
        reg_lambda = trial.suggest_float('reg_lambda', 1e-2, 1e0, log=True)
        objective = 'multiclass'
        boosting_type = trial.suggest_categorical('boosting_type', ['gbdt'])
        device_type = trial.suggest_categorical('device_type', ['gpu'])
        max_bin = trial.suggest_int('max_bin', 63, 255)
        bagging_freq = 1 
        model = LGBMClassifier(
          n_estimators=n_estimators,
          learning_rate=learning_rate,
          num_leaves=num_leaves,
          subsample=bagging_fraction,        
          colsample_bytree=feature_fraction, 
          reg_alpha=reg_alpha,
          reg_lambda=reg_lambda,
          objective=objective,
          boosting_type=boosting_type,
          device=device_type,
          random_state=42,
          max_bin=max_bin,
          subsample_freq=bagging_freq
)
    else:
        raise ValueError("Model name must be 'RandomForest', 'XGBoost', or 'LightGBM'")

# 3-Fold Stratified Cross-Validation is used to evaluate the model's performance with average F1-Macro score
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    score = cross_val_score(model, X, y, scoring='f1_macro', cv=skf, n_jobs=-1).mean()
    return score


# Optuna's HyperbandPruner for faster convergence
pruner = HyperbandPruner(min_resource=1, max_resource="auto", reduction_factor=3)

In [ ]:
# Since the dataset is large, we can use a small subset of the training data for hyperparameter optimization to speed up the process
# Subsampled dataset would represent the whole dataset, however, the scores might be slightly lower than using the full dataset as significant information is lost to capture the relationships
# Yet, it is a good trade-off between speed and performance for hyperparameter 
# Eventually, the model optimized will be trained on the full data which in return will improve the f1-macro scores
# I will use 5% of the training data (27,400) for hyperparameter optimization, and will enable GPU usage for further speed

X_subsample, _ , y_subsample, _ = train_test_split(X_train, y_train, train_size=0.05, stratify=y_train, random_state=42)

# Reqyuired for LightGBM to treat categorical features appropriately
X_subsample_lgb = X_subsample.copy()
X_subsample_lgb[['Soil_Type_Cat'] + wilderness_cols] = X_subsample_lgb[['Soil_Type_Cat']+ wilderness_cols].astype("category")

# Creating a dictionary to hold model configurations
model_configs = {
    'LightGBM': {'data': X_subsample_lgb},
    'XGBoost': {'data': X_subsample},
    'RandomForest': {'data': X_subsample}
}

# Number of Trials for each model optimization (similar to epochs in neural networks)
n_trials = 25

# Recording best trials for each model
best_trials = {}



In [ ]:
# Running optimization for all models
for model_name, config in model_configs.items():
    print(f'\n\nHyperparameter Optimization for {model_name}')
    print('=' * 150)
    study = optuna.create_study(
        direction='maximize', 
        pruner=pruner,
        storage="sqlite:///optuna_ordinal_v2.db",      # SQLite database to store study results
        study_name=f"{model_name}_ordinal_study_v2",   # Unique study name for each model
        load_if_exists=True                 # To continue from previous trials if they exist
    )

    # Objective function required by Optuna
    def model_objective(trial):
        return objective(trial, model_name, config['data'], y_subsample)
    
    study.optimize(
        model_objective, 
        n_trials=n_trials, 
        show_progress_bar=True
    )
    
    print(f'\nBest F1-macro: {study.best_value:.4f}')
    print('Best hyperparameters:', study.best_params)
    best_trials[model_name] = study.best_params


# Although the trials are shown as 25, the actual number of total trials is higher due to previously completed trials under the same study_name in the same optuna_ordinal_v2.db database
# Hence, it is possible that the best hyperparameters might be from any of the previous trials
# Optuna's search algorithm is benefitted from the previous trials and uses that information to find better hyperparameters in the next trial(s)
# For this case, I haven't changed any hyperparameter ranges or the number of trials between runs, so it is like continuing the same optimization process
# As a matter of fact, I forgot to change the number of trials in the previous runs :D, therefore, it is like I did partially add trials to previous runs
# But it is technically possible to add/remove hyperparameters safely between runs as long as the study_name remains the same and load_if_exists is set to True
# However, it would be problematic to change the search range as it would mislead the search algorithm and the results would be unreliable


In [ ]:
model_lgbm = LGBMClassifier(
          **best_trials['LightGBM'],    # Unpacking the best parameters
          objective='multiclass',
          random_state=42,
          subsample_freq=1,             # Required when bagging_fraction < 1.0
          verbose=-1                    
)

model_xgbo = XGBClassifier(
          **best_trials['XGBoost'],     # Unpacking the best parameters
          objective='multiclass',
          random_state=42,
          subsample_freq=1,             # Required when bagging_fraction < 1.0
)

model_rf = RandomForestClassifier(
          **best_trials['RandomForest'], # Unpacking the best parameters
          random_state=42
)

# 6. Results
---

In [ ]:
# Transformation of categorical features for LightGBM
X_train_categorical = X_train.copy()
X_train_categorical[['Soil_Type_Cat'] + wilderness_cols] = X_train_categorical[['Soil_Type_Cat'] + wilderness_cols].astype("category")

models = {'LightGBM': model_lgbm, 'XGBoost': model_xgbo, 'RandomForest': model_rf}
model_performance = {}
y_pred_dict = {}

for name, model in models.items():
    print(f'Fitting {name}...')
    if name == 'LightGBM':
        model.fit(X_train_categorical, y_train)
        # Transformation of categorical features for LightGBM test set
        X_test_categorical = X_test.copy()
        X_test_categorical[['Soil_Type_Cat'] + wilderness_cols] = X_test_categorical[['Soil_Type_Cat'] + wilderness_cols].astype("category")
        y_pred_dict[name] = model.predict(X_test_categorical)
    else:
        model.fit(X_train, y_train)
        y_pred_dict[name] = model.predict(X_test)

In [ ]:
# F1-Macro Score Evaluation
for name, y_pred in y_pred_dict.items():
    f1_macro_test = f1_score(y_test, y_pred, average='macro')
    model_performance[name] = f1_macro_test
    print(f"F1-Macro Score for {name}: {f1_macro_test:.4f}\n")

# I chose F1-Macro score as the evaluation metric because the dataset is imbalanced and it is a multi-class classification problem
# It seems that all performed above 0.90 which is a competitive score
# Among all modes, XGBoost performed the best with a F1-Macro score

In [ ]:
y_pred_xgbo= y_pred_dict['XGBoost']  

# XGBoost Classification Report
print(classification_report(y_test, y_pred_xgbo))

# As a final evaluation, I will print the classification report for the best performing model, which is XGBoost in this case (Other models are also competitive!)
# Since class 3 and class 4 are the least frequent classes, their precision and recall are slightly lower than other classes, which is expected in imbalanced datasets, however, they are still quite good (> 0.90)

In [ ]:
from xgboost import plot_importance

plot_importance(model_xgbo, importance_type='gain', title='Top 10 Feature Importances (XGBoost)', xlabel='Importance Score', ylabel='Features',values_format="{v:.2f}", grid=False)
plt.show()

# As it can be seen Wilderness Area, Soil Type, and Elevation are among the most important features in determining the Cover Type
# This aligns with our earlier observations during the exploratory data analysis (EDA) phase, where we noted the significance of these features in relation to the target variable.